In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast 
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, evaluate
import warnings; warnings.simplefilter('ignore')
import pyodbc

In [2]:
server = ''database = ''username = ''password = ''driver='/usr/local/lib/libmsodbcsql.13.dylib'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [7]:
metaData = pd.DataFrame()
query_meta = "SELECT CONVERT(int,CONVERT(float,id)) AS id, title, release_date, genres, tagline, overview, imdb_id, popularity, status, vote_average, vote_count FROM [dbo].[movie_metadata] "
for chunk in pd.read_sql_query(query_meta, cnxn, chunksize=10**4):
    metaData = pd.concat([metaData, chunk])
    print(metaData)
print(metaData.shape)

          id                                 title release_date  \
0        862                             Toy Story   1995-10-30   
1       8844                               Jumanji   1995-12-15   
2      15602                      Grumpier Old Men   1995-12-22   
3      31357                     Waiting to Exhale   1995-12-22   
4     253632                           Baton Rouge   1988-10-08   
5      79048                          C'est la vie   2001-11-07   
6      11862           Father of the Bride Part II   1995-02-10   
7        949                                  Heat   1995-12-15   
8      11860                               Sabrina   1995-12-15   
9      45325                          Tom and Huck   1995-12-22   
10      9091                          Sudden Death   1995-12-22   
11       710                             GoldenEye   1995-11-16   
12      9087                The American President   1995-11-17   
13     12110           Dracula: Dead and Loving It   1995-12-2

          id                                              title release_date  \
0        862                                          Toy Story   1995-10-30   
1       8844                                            Jumanji   1995-12-15   
2      15602                                   Grumpier Old Men   1995-12-22   
3      31357                                  Waiting to Exhale   1995-12-22   
4     253632                                        Baton Rouge   1988-10-08   
5      79048                                       C'est la vie   2001-11-07   
6      11862                        Father of the Bride Part II   1995-02-10   
7        949                                               Heat   1995-12-15   
8      11860                                            Sabrina   1995-12-15   
9      45325                                       Tom and Huck   1995-12-22   
10      9091                                       Sudden Death   1995-12-22   
11       710                            

          id                              title release_date  \
0        862                          Toy Story   1995-10-30   
1       8844                            Jumanji   1995-12-15   
2      15602                   Grumpier Old Men   1995-12-22   
3      31357                  Waiting to Exhale   1995-12-22   
4     253632                        Baton Rouge   1988-10-08   
5      79048                       C'est la vie   2001-11-07   
6      11862        Father of the Bride Part II   1995-02-10   
7        949                               Heat   1995-12-15   
8      11860                            Sabrina   1995-12-15   
9      45325                       Tom and Huck   1995-12-22   
10      9091                       Sudden Death   1995-12-22   
11       710                          GoldenEye   1995-11-16   
12      9087             The American President   1995-11-17   
13     12110        Dracula: Dead and Loving It   1995-12-22   
14     21032                            

          id                           title release_date  \
0        862                       Toy Story   1995-10-30   
1       8844                         Jumanji   1995-12-15   
2      15602                Grumpier Old Men   1995-12-22   
3      31357               Waiting to Exhale   1995-12-22   
4     253632                     Baton Rouge   1988-10-08   
5      79048                    C'est la vie   2001-11-07   
6      11862     Father of the Bride Part II   1995-02-10   
7        949                            Heat   1995-12-15   
8      11860                         Sabrina   1995-12-15   
9      45325                    Tom and Huck   1995-12-22   
10      9091                    Sudden Death   1995-12-22   
11       710                       GoldenEye   1995-11-16   
12      9087          The American President   1995-11-17   
13     12110     Dracula: Dead and Loving It   1995-12-22   
14     21032                           Balto   1995-12-22   
15     10858            

          id                             title release_date  \
0        862                         Toy Story   1995-10-30   
1       8844                           Jumanji   1995-12-15   
2      15602                  Grumpier Old Men   1995-12-22   
3      31357                 Waiting to Exhale   1995-12-22   
4     253632                       Baton Rouge   1988-10-08   
5      79048                      C'est la vie   2001-11-07   
6      11862       Father of the Bride Part II   1995-02-10   
7        949                              Heat   1995-12-15   
8      11860                           Sabrina   1995-12-15   
9      45325                      Tom and Huck   1995-12-22   
10      9091                      Sudden Death   1995-12-22   
11       710                         GoldenEye   1995-11-16   
12      9087            The American President   1995-11-17   
13     12110       Dracula: Dead and Loving It   1995-12-22   
14     21032                             Balto   1995-1

In [8]:
credits = pd.DataFrame()
query_credits = "SELECT * FROM [dbo].[credits]"
for chunk in pd.read_sql_query(query_credits, cnxn, chunksize=10**4):
    credits = pd.concat([credits, chunk])
    print(credits)
print(credits.shape)

                                                   cast  \
0     [{'cast_id': 1, 'character': 'Maj. Jock Sincla...   
1     [{'cast_id': 1, 'character': 'Louis', 'credit_...   
2     [{'cast_id': 1, 'character': 'Martha Travis', ...   
3     [{'cast_id': 1, 'character': 'Liz', 'credit_id...   
4     [{'cast_id': 2, 'character': 'Francis Sullivan...   
5     [{'cast_id': 4, 'character': 'Glaeken Trismege...   
6     [{'cast_id': 5, 'character': 'Commercial Photo...   
7     [{'cast_id': 5, 'character': 'Joji Atsumi', 'c...   
8     [{'cast_id': 1, 'character': 'Joe Kidd', 'cred...   
9     [{'cast_id': 1, 'character': 'Stuart Sawyer', ...   
10    [{'cast_id': 1, 'character': 'Neal Downs', 'cr...   
11    [{'cast_id': 5, 'character': 'Mr. Franz', 'cre...   
12    [{'cast_id': 4, 'character': 'Abhay', 'credit_...   
13    [{'cast_id': 2, 'character': 'Lady Edwina Eske...   
14    [{'cast_id': 3, 'character': 'Elaine Cheng', '...   
15    [{'cast_id': 1, 'character': 'Professor Tom Ne... 

                                                   cast  \
0     [{'cast_id': 1, 'character': 'Maj. Jock Sincla...   
1     [{'cast_id': 1, 'character': 'Louis', 'credit_...   
2     [{'cast_id': 1, 'character': 'Martha Travis', ...   
3     [{'cast_id': 1, 'character': 'Liz', 'credit_id...   
4     [{'cast_id': 2, 'character': 'Francis Sullivan...   
5     [{'cast_id': 4, 'character': 'Glaeken Trismege...   
6     [{'cast_id': 5, 'character': 'Commercial Photo...   
7     [{'cast_id': 5, 'character': 'Joji Atsumi', 'c...   
8     [{'cast_id': 1, 'character': 'Joe Kidd', 'cred...   
9     [{'cast_id': 1, 'character': 'Stuart Sawyer', ...   
10    [{'cast_id': 1, 'character': 'Neal Downs', 'cr...   
11    [{'cast_id': 5, 'character': 'Mr. Franz', 'cre...   
12    [{'cast_id': 4, 'character': 'Abhay', 'credit_...   
13    [{'cast_id': 2, 'character': 'Lady Edwina Eske...   
14    [{'cast_id': 3, 'character': 'Elaine Cheng', '...   
15    [{'cast_id': 1, 'character': 'Professor Tom Ne... 

                                                   cast  \
0     [{'cast_id': 1, 'character': 'Maj. Jock Sincla...   
1     [{'cast_id': 1, 'character': 'Louis', 'credit_...   
2     [{'cast_id': 1, 'character': 'Martha Travis', ...   
3     [{'cast_id': 1, 'character': 'Liz', 'credit_id...   
4     [{'cast_id': 2, 'character': 'Francis Sullivan...   
5     [{'cast_id': 4, 'character': 'Glaeken Trismege...   
6     [{'cast_id': 5, 'character': 'Commercial Photo...   
7     [{'cast_id': 5, 'character': 'Joji Atsumi', 'c...   
8     [{'cast_id': 1, 'character': 'Joe Kidd', 'cred...   
9     [{'cast_id': 1, 'character': 'Stuart Sawyer', ...   
10    [{'cast_id': 1, 'character': 'Neal Downs', 'cr...   
11    [{'cast_id': 5, 'character': 'Mr. Franz', 'cre...   
12    [{'cast_id': 4, 'character': 'Abhay', 'credit_...   
13    [{'cast_id': 2, 'character': 'Lady Edwina Eske...   
14    [{'cast_id': 3, 'character': 'Elaine Cheng', '...   
15    [{'cast_id': 1, 'character': 'Professor Tom Ne... 

In [9]:
keywords = pd.DataFrame()
query_keywords = "SELECT * FROM [dbo].[keywords]"
for chunk in pd.read_sql_query(query_keywords, cnxn, chunksize=10**4):
    keywords = pd.concat([keywords, chunk])
    print(keywords)
print(keywords.shape)

          id                                           keywords
0        862  [{'id': 931, 'name': 'jealousy'}, {'id': 4290,...
1       8844  [{'id': 10090, 'name': 'board game'}, {'id': 1...
2      15602  [{'id': 1495, 'name': 'fishing'}, {'id': 12392...
3      31357  [{'id': 818, 'name': 'based on novel'}, {'id':...
4      11862  [{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...
5        949  [{'id': 642, 'name': 'robbery'}, {'id': 703, '...
6      11860  [{'id': 90, 'name': 'paris'}, {'id': 380, 'nam...
7      45325                                                 []
8       9091  [{'id': 949, 'name': 'terrorist'}, {'id': 1562...
9        710  [{'id': 701, 'name': 'cuba'}, {'id': 769, 'nam...
10      9087  [{'id': 833, 'name': 'white house'}, {'id': 84...
11     12110  [{'id': 3633, 'name': 'dracula'}, {'id': 11931...
12     21032  [{'id': 1994, 'name': 'wolf'}, {'id': 6411, 'n...
13     33689  [{'id': 13014, 'name': 'orphan'}, {'id': 20435...
14     10858  [{'id': 840, 'name': 'usa 

          id                                           keywords
0        862  [{'id': 931, 'name': 'jealousy'}, {'id': 4290,...
1       8844  [{'id': 10090, 'name': 'board game'}, {'id': 1...
2      15602  [{'id': 1495, 'name': 'fishing'}, {'id': 12392...
3      31357  [{'id': 818, 'name': 'based on novel'}, {'id':...
4      11862  [{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...
5        949  [{'id': 642, 'name': 'robbery'}, {'id': 703, '...
6      11860  [{'id': 90, 'name': 'paris'}, {'id': 380, 'nam...
7      45325                                                 []
8       9091  [{'id': 949, 'name': 'terrorist'}, {'id': 1562...
9        710  [{'id': 701, 'name': 'cuba'}, {'id': 769, 'nam...
10      9087  [{'id': 833, 'name': 'white house'}, {'id': 84...
11     12110  [{'id': 3633, 'name': 'dracula'}, {'id': 11931...
12     21032  [{'id': 1994, 'name': 'wolf'}, {'id': 6411, 'n...
13     33689  [{'id': 13014, 'name': 'orphan'}, {'id': 20435...
14     10858  [{'id': 840, 'name': 'usa 

In [10]:
links = pd.DataFrame()
query_links = "SELECT * FROM [dbo].[links]"
for chunk in pd.read_sql_query(query_links, cnxn, chunksize=10**4):
    links = pd.concat([links, chunk])
    print(links)
print(links.shape)

      movieId  imdbId   tmdbId
0           1  114709    862.0
1           2  113497   8844.0
2           3  113228  15602.0
3           4  114885  31357.0
4           5  113041  11862.0
5           6  113277    949.0
6           7  114319  11860.0
7           8  112302  45325.0
8           9  114576   9091.0
9          10  113189    710.0
10         11  112346   9087.0
11         12  112896  12110.0
12         13  112453  21032.0
13         14  113987  10858.0
14         15  112760   1408.0
15         16  112641    524.0
16         17  114388   4584.0
17         18  113101      5.0
18         19  112281   9273.0
19         20  113845  11517.0
20         21  113161   8012.0
21         22  112722   1710.0
22         23  112401   9691.0
23         24  114168  12665.0
24         25  113627    451.0
25         26  114057  16420.0
26         27  114011   9263.0
27         28  114117  17015.0
28         29  112682    902.0
29         30  115012  37557.0
...       ...     ...      ...
9970    

In [12]:
print(credits.dtypes)
print(keywords.dtypes)
print(links.dtypes)
print(metaData.dtypes)

cast    object
crew    object
id       int64
dtype: object
id           int64
keywords    object
dtype: object
movieId      int64
imdbId       int64
tmdbId     float64
dtype: object
id               int64
title           object
release_date    object
genres          object
tagline         object
overview        object
imdb_id         object
popularity      object
status          object
vote_average    object
vote_count      object
dtype: object


In [13]:
metaData['year'] = pd.to_datetime(metaData['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
metaData['year'].replace({'NaT': '0000'}, inplace=True)
metaData[['year']] = metaData[['year']].astype(int)

In [14]:
metaData.head(5)

,id,title,release_date,genres,tagline,overview,imdb_id,popularity,status,vote_average,vote_count,year
0,862,Toy Story,1995-10-30,"['Animation', 'Comedy', 'Family']",None,"Led by Woody, Andy's toys live happily in his ...",tt0114709,21.946943,Released,7.7,5415.0,1995
1,8844,Jumanji,1995-12-15,"['Adventure', 'Fantasy', 'Family']",Roll the dice and unleash the excitement!,When siblings Judy and Peter discover an encha...,tt0113497,17.015539,Released,6.9,2413.0,1995
2,15602,Grumpier Old Men,1995-12-22,"['Romance', 'Comedy']",Still Yelling. Still Fighting. Still Ready for...,A family wedding reignites the ancient feud be...,tt0113228,11.7129,Released,6.5,92.0,1995
3,31357,Waiting to Exhale,1995-12-22,"['Comedy', 'Drama', 'Romance']",Friends are the people who let you be yourself...,"Cheated on, mistreated and stepped on, the wom...",tt0114885,3.859495,Released,6.1,34.0,1995
4,253632,Baton Rouge,1988-10-08,['Thriller'],None,None,tt0094822,0.437895,Released,0.0,0.0,1988


In [15]:
metaData['flag'] = metaData['year'].apply(lambda x: 1 if x > 2015 else 0)

In [16]:
metaData

,id,title,release_date,genres,tagline,overview,imdb_id,popularity,status,vote_average,vote_count,year,flag
0,862,Toy Story,1995-10-30,"['Animation', 'Comedy', 'Family']",None,"Led by Woody, Andy's toys live happily in his ...",tt0114709,21.946943,Released,7.7,5415.0,1995,0
1,8844,Jumanji,1995-12-15,"['Adventure', 'Fantasy', 'Family']",Roll the dice and unleash the excitement!,When siblings Judy and Peter discover an encha...,tt0113497,17.015539,Released,6.9,2413.0,1995,0
2,15602,Grumpier Old Men,1995-12-22,"['Romance', 'Comedy']",Still Yelling. Still Fighting. Still Ready for...,A family wedding reignites the ancient feud be...,tt0113228,11.7129,Released,6.5,92.0,1995,0
3,31357,Waiting to Exhale,1995-12-22,"['Comedy', 'Drama', 'Romance']",Friends are the people who let you be yourself...,"Cheated on, mistreated and stepped on, the wom...",tt0114885,3.859495,Released,6.1,34.0,1995,0
4,253632,Baton Rouge,1988-10-08,['Thriller'],None,None,tt0094822,0.437895,Released,0.0,0.0,1988,0
5,79048,C'est la vie,2001-11-07,"['Drama', 'Comedy']",None,None,tt0260198,0.001714,Released,0.0,0.0,2001,0
6,11862,Father of the Bride Part II,1995-02-10,['Comedy'],Just When His World Is Back To Normal... He's ...,Just when George Banks has recovered from his ...,tt0113041,8.387519000000001,Released,5.7,173.0,1995,0
7,949,Heat,1995-12-15,"['Action', 'Crime', 'Drama', 'Thriller']",A Los Angeles Crime Saga,"Obsessive master thief, Neil McCauley leads a ...",tt0113277,17.924927,Released,7.7,1886.0,1995,0
8,11860,Sabrina,1995-12-15,"['Comedy', 'Romance']",You are cordially invited to the most surprisi...,An ugly duckling having undergone a remarkable...,tt0114319,6.677277,Released,6.2,141.0,1995,0
9,45325,Tom and Huck,1995-12-22,"['Action', 'Adventure', 'Drama', 'Family']",The Original Bad Boys.,"A mischievous young boy, Tom Sawyer, witnesses...",tt0112302,2.5611610000000002,Released,5.4,45.0,1995,0


In [17]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metaData['id'] = metaData['id'].astype('int')
links = links[links['tmdbId'].notnull()]['tmdbId'].astype('int')

In [18]:
metaData

,id,title,release_date,genres,tagline,overview,imdb_id,popularity,status,vote_average,vote_count,year,flag
0,862,Toy Story,1995-10-30,"['Animation', 'Comedy', 'Family']",None,"Led by Woody, Andy's toys live happily in his ...",tt0114709,21.946943,Released,7.7,5415.0,1995,0
1,8844,Jumanji,1995-12-15,"['Adventure', 'Fantasy', 'Family']",Roll the dice and unleash the excitement!,When siblings Judy and Peter discover an encha...,tt0113497,17.015539,Released,6.9,2413.0,1995,0
2,15602,Grumpier Old Men,1995-12-22,"['Romance', 'Comedy']",Still Yelling. Still Fighting. Still Ready for...,A family wedding reignites the ancient feud be...,tt0113228,11.7129,Released,6.5,92.0,1995,0
3,31357,Waiting to Exhale,1995-12-22,"['Comedy', 'Drama', 'Romance']",Friends are the people who let you be yourself...,"Cheated on, mistreated and stepped on, the wom...",tt0114885,3.859495,Released,6.1,34.0,1995,0
4,253632,Baton Rouge,1988-10-08,['Thriller'],None,None,tt0094822,0.437895,Released,0.0,0.0,1988,0
5,79048,C'est la vie,2001-11-07,"['Drama', 'Comedy']",None,None,tt0260198,0.001714,Released,0.0,0.0,2001,0
6,11862,Father of the Bride Part II,1995-02-10,['Comedy'],Just When His World Is Back To Normal... He's ...,Just when George Banks has recovered from his ...,tt0113041,8.387519000000001,Released,5.7,173.0,1995,0
7,949,Heat,1995-12-15,"['Action', 'Crime', 'Drama', 'Thriller']",A Los Angeles Crime Saga,"Obsessive master thief, Neil McCauley leads a ...",tt0113277,17.924927,Released,7.7,1886.0,1995,0
8,11860,Sabrina,1995-12-15,"['Comedy', 'Romance']",You are cordially invited to the most surprisi...,An ugly duckling having undergone a remarkable...,tt0114319,6.677277,Released,6.2,141.0,1995,0
9,45325,Tom and Huck,1995-12-22,"['Action', 'Adventure', 'Drama', 'Family']",The Original Bad Boys.,"A mischievous young boy, Tom Sawyer, witnesses...",tt0112302,2.5611610000000002,Released,5.4,45.0,1995,0


In [19]:
metaData = metaData.merge(credits, on='id')
metaData = metaData.merge(keywords, on='id')

In [20]:
metaData

,id,title,release_date,genres,tagline,overview,imdb_id,popularity,status,vote_average,vote_count,year,flag,cast,crew,keywords
0,862,Toy Story,1995-10-30,"['Animation', 'Comedy', 'Family']",None,"Led by Woody, Andy's toys live happily in his ...",tt0114709,21.946943,Released,7.7,5415.0,1995,0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,Jumanji,1995-12-15,"['Adventure', 'Fantasy', 'Family']",Roll the dice and unleash the excitement!,When siblings Judy and Peter discover an encha...,tt0113497,17.015539,Released,6.9,2413.0,1995,0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,Grumpier Old Men,1995-12-22,"['Romance', 'Comedy']",Still Yelling. Still Fighting. Still Ready for...,A family wedding reignites the ancient feud be...,tt0113228,11.7129,Released,6.5,92.0,1995,0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,Waiting to Exhale,1995-12-22,"['Comedy', 'Drama', 'Romance']",Friends are the people who let you be yourself...,"Cheated on, mistreated and stepped on, the wom...",tt0114885,3.859495,Released,6.1,34.0,1995,0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,253632,Baton Rouge,1988-10-08,['Thriller'],None,None,tt0094822,0.437895,Released,0.0,0.0,1988,0,"[{'cast_id': 1, 'character': 'Ana Alonzo', 'cr...",[],[]
5,79048,C'est la vie,2001-11-07,"['Drama', 'Comedy']",None,None,tt0260198,0.001714,Released,0.0,0.0,2001,0,"[{'cast_id': 2, 'character': 'Dimitri', 'credi...","[{'credit_id': '52fe49c0c3a368484e13d8db', 'de...",[]
6,11862,Father of the Bride Part II,1995-02-10,['Comedy'],Just When His World Is Back To Normal... He's ...,Just when George Banks has recovered from his ...,tt0113041,8.387519000000001,Released,5.7,173.0,1995,0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
7,949,Heat,1995-12-15,"['Action', 'Crime', 'Drama', 'Thriller']",A Los Angeles Crime Saga,"Obsessive master thief, Neil McCauley leads a ...",tt0113277,17.924927,Released,7.7,1886.0,1995,0,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...","[{'id': 642, 'name': 'robbery'}, {'id': 703, '..."
8,11860,Sabrina,1995-12-15,"['Comedy', 'Romance']",You are cordially invited to the most surprisi...,An ugly duckling having undergone a remarkable...,tt0114319,6.677277,Released,6.2,141.0,1995,0,"[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de...","[{'id': 90, 'name': 'paris'}, {'id': 380, 'nam..."
9,45325,Tom and Huck,1995-12-22,"['Action', 'Adventure', 'Drama', 'Family']",The Original Bad Boys.,"A mischievous young boy, Tom Sawyer, witnesses...",tt0112302,2.5611610000000002,Released,5.4,45.0,1995,0,"[{'cast_id': 2, 'character': 'Tom Sawyer', 'cr...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",[]


In [21]:
metaDataLinks = metaData[metaData.id.isin(links)]

In [22]:
metaDataLinks

,id,title,release_date,genres,tagline,overview,imdb_id,popularity,status,vote_average,vote_count,year,flag,cast,crew,keywords
0,862,Toy Story,1995-10-30,"['Animation', 'Comedy', 'Family']",None,"Led by Woody, Andy's toys live happily in his ...",tt0114709,21.946943,Released,7.7,5415.0,1995,0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,Jumanji,1995-12-15,"['Adventure', 'Fantasy', 'Family']",Roll the dice and unleash the excitement!,When siblings Judy and Peter discover an encha...,tt0113497,17.015539,Released,6.9,2413.0,1995,0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,Grumpier Old Men,1995-12-22,"['Romance', 'Comedy']",Still Yelling. Still Fighting. Still Ready for...,A family wedding reignites the ancient feud be...,tt0113228,11.7129,Released,6.5,92.0,1995,0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...","[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,Waiting to Exhale,1995-12-22,"['Comedy', 'Drama', 'Romance']",Friends are the people who let you be yourself...,"Cheated on, mistreated and stepped on, the wom...",tt0114885,3.859495,Released,6.1,34.0,1995,0,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...","[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,253632,Baton Rouge,1988-10-08,['Thriller'],None,None,tt0094822,0.437895,Released,0.0,0.0,1988,0,"[{'cast_id': 1, 'character': 'Ana Alonzo', 'cr...",[],[]
6,11862,Father of the Bride Part II,1995-02-10,['Comedy'],Just When His World Is Back To Normal... He's ...,Just when George Banks has recovered from his ...,tt0113041,8.387519000000001,Released,5.7,173.0,1995,0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...","[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
7,949,Heat,1995-12-15,"['Action', 'Crime', 'Drama', 'Thriller']",A Los Angeles Crime Saga,"Obsessive master thief, Neil McCauley leads a ...",tt0113277,17.924927,Released,7.7,1886.0,1995,0,"[{'cast_id': 25, 'character': 'Lt. Vincent Han...","[{'credit_id': '52fe4292c3a36847f802916d', 'de...","[{'id': 642, 'name': 'robbery'}, {'id': 703, '..."
8,11860,Sabrina,1995-12-15,"['Comedy', 'Romance']",You are cordially invited to the most surprisi...,An ugly duckling having undergone a remarkable...,tt0114319,6.677277,Released,6.2,141.0,1995,0,"[{'cast_id': 1, 'character': 'Linus Larrabee',...","[{'credit_id': '52fe44959251416c75039da9', 'de...","[{'id': 90, 'name': 'paris'}, {'id': 380, 'nam..."
9,45325,Tom and Huck,1995-12-22,"['Action', 'Adventure', 'Drama', 'Family']",The Original Bad Boys.,"A mischievous young boy, Tom Sawyer, witnesses...",tt0112302,2.5611610000000002,Released,5.4,45.0,1995,0,"[{'cast_id': 2, 'character': 'Tom Sawyer', 'cr...","[{'credit_id': '52fe46bdc3a36847f810f797', 'de...",[]
10,9091,Sudden Death,1995-12-22,"['Action', 'Adventure', 'Thriller']",Terror goes into overtime.,International action superstar Jean Claude Van...,tt0114576,5.23158,Released,5.5,174.0,1995,0,"[{'cast_id': 1, 'character': 'Darren Francis T...","[{'credit_id': '52fe44dbc3a36847f80ae0f1', 'de...","[{'id': 949, 'name': 'terrorist'}, {'id': 1562..."


In [23]:
links

0          862
1         8844
2        15602
3        31357
4        11862
5          949
6        11860
7        45325
8         9091
9          710
10        9087
11       12110
12       21032
13       10858
14        1408
15         524
16        4584
17           5
18        9273
19       11517
20        8012
21        1710
22        9691
23       12665
24         451
25       16420
26        9263
27       17015
28         902
29       37557
         ...  
7248      9496
7249     73770
7250     10036
7251    233438
7252     11664
7253      9572
7254      1915
7255     51880
7256    258751
7257    208982
7258     56771
7259    254578
7260    328595
7261    160171
7262    287422
7263    254200
7264    293771
7265    301225
7266     35428
7267     91628
7268      9478
7269     10470
7270     10010
7271      9565
7272      9677
7273      4436
7274      9274
7275    285213
7276     32099
7277    286971
Name: tmdbId, Length: 27026, dtype: int64